# Estimation and Hypothesis Testing

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.5
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
options(repr.plot.width=4, repr.plot.height=3)

## Set random number seed for reproucibility

In [3]:
set.seed(42)

## Point estimates

In [4]:
x <- rnorm(10)

In [5]:
x

[1]  1.37095845 -0.56469817  0.36312841  0.63286260  0.40426832 -0.10612452
 [7]  1.51152200 -0.09465904  2.01842371 -0.06271410

### Mean

Manual calculation

In [6]:
sum(x)/length(x)

[1] 0.5472968

Using built-in function

In [7]:
mean(x)

[1] 0.5472968

### Median

Manual calculation

In [8]:
x_sorted <- sort(x)

In [9]:
length(x)

[1] 10

Since there are an even number of observations, we need the average of the middle two data poitns

In [10]:
sum(x_sorted[5:6])/2

[1] 0.3836984

Using built-in function

In [11]:
median(x)

[1] 0.3836984

### Quantiles

The mean is just the 50 percentile. We can use R to get any percentile we like.

In [12]:
quantile(x, 0.5)

50% 
0.3836984

In [13]:
quantile(x, seq(0,1,length.out = 5))

0%        25%        50%        75%       100% 
-0.5646982 -0.0866728  0.3836984  1.1864345  2.0184237

## Interval estimates

### Confidence intervals

In [23]:
ci = 0.95

In [24]:
alpha = (1-ci)
n <- length(x)
m <- mean(x)
s <- sd(x)
se <- s/sqrt(n)
me <- qt(1-alpha/2, df=n-1) * se
c(m - me, m + me)

[1] -0.05034732  1.14494085

Note that confidence intervals get larger as the confidence required increases.

In [25]:
ci = 0.99

In [26]:
alpha = (1-ci)
n <- length(x)
m <- mean(x)
s <- sd(x)
se <- s/sqrt(n)
me <- qt(1-alpha/2, df=n-1) * se
c(m - me, m + me)

[1] -0.3112842  1.4058777

### Making a function

In [27]:
conf <- function(x, ci=0.95) {
   alpha = (1-ci)
    n <- length(x)
    m <- mean(x)
    s <- sd(x)
    se <- s/sqrt(n)
    me <- qt(1-alpha/2, df=n-1) * se
    c(m - me, m + me)     
}

In [28]:
conf(x)

[1] -0.05034732  1.14494085

### Coverage

In 1,000 experiments, we expect the true mean (0) to lie within the estimated 95% CIs 950 times.

In [46]:
n_expt <- 1000
n <- 10
cls <- t(replicate(n_expt, conf(rnorm(n))))

In [47]:
sum(cls[,1] < 0 & 0 < cls[,2])

[1] 955

## Hypothesis testing